In [2]:
# Dependencies and Setup
import pandas as pd
import matplotlib as plt
import numpy as np
from datetime import datetime
from scipy import stats
from matplotlib import pyplot

In [3]:
# Import the requests library
import requests

# Import the API key
from config import weather_api_key

In [4]:
# Starting URL for WeatherMap API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=5cc9ad261292a20b92a7bc8c2d2ac550


In [5]:
# Create a new set of random latitude and longitude combinations for 2000 random cities
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [6]:
# Get the nearest city using the citpy module
from citipy import citipy

In [7]:
# Create a list to hold all of the cities
cities = []

# Identify the nearest city for each latitude and longitutde combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficent count
len(cities)

741

In [8]:
# Create an empty list to hold all the weather data for the cities
city_data = []

# Print the beginning of the logging process
print("Beginning Data Retrieval    ")
print("----------------------------")

# Create counters
record_count = 1
set_count = 1

Beginning Data Retrieval    
----------------------------


In [ ]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        weather_description = city_weather["weather"][0]["description"]
        
        try:
            rain_inches = city_weather["rain"]["3h"]
        except:
            rain_inches = ("0")
            pass
        try:
            snow_inches = city_weather["snow"]["3h"]
        except:
            snow_inches = ("0")  
            pass
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": weather_description,
                          "Rain inches (last 3 hours)": rain_inches,
                          "Snow inches (last 3 hours)": snow_inches})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | butaritari
Processing Record 2 of Set 1 | broome
Processing Record 3 of Set 1 | hermanus
Processing Record 4 of Set 1 | punta arenas
Processing Record 5 of Set 1 | agadir
Processing Record 6 of Set 1 | lluta
Processing Record 7 of Set 1 | uruzgan
Processing Record 8 of Set 1 | madang
Processing Record 9 of Set 1 | evensk
Processing Record 10 of Set 1 | awjilah
Processing Record 11 of Set 1 | saint-joseph
Processing Record 12 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | aden
Processing Record 15 of Set 1 | mao
Processing Record 16 of Set 1 | bayir
Processing Record 17 of Set 1 | bredasdorp
Processing Record 18 of Set 1 | chokurdakh
Processing Record 19 of Set 1 | barrow
Processing Record 20 of Set 1 | pierre
Processing Record 21 of Set 1 | nyimba
Processing Record 22 of Set 1 | saint-philippe
Processing Record 23 of Set 1 | port alfred
Processing Record 24 of Set 1 | upernavik
Pr

In [ ]:
# Add the data to a new dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df

In [ ]:
# Create the output file (CSV).
output_data_file = "Vacation_Search/Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")